# Сравнение моделей

In [52]:
from catboost import CatBoostClassifier
import optuna
import pandas as pd
import wandb
from sklearn.metrics import f1_score, precision_score, recall_score, balanced_accuracy_score, accuracy_score

In [13]:
!wandb login

In [14]:
models_path = [
    'catboost_baseline.model',
    'catboost_full.model',
    'catboost_with_text.model',
    'catboost_with_id.model',
    'catboost_with_text-class.model',
    'catboost_with_text_text-class.model',
    'catboost_with_text.model'
]

In [20]:
X_train = pd.read_csv('../data/unbalanced_data/train_data.csv')
Y_train = X_train['label']

X_valid = pd.read_csv('../data/unbalanced_data/valid_data.csv')
Y_valid = X_valid['label']

X_test = pd.read_csv('../data/unbalanced_data/test_data.csv')
Y_test = X_test['label']

In [21]:
%env WANDB_SILENT=True

env: WANDB_SILENT=True


In [39]:
for path in models_path:
    full_path = r"./"+path
    print(full_path)
    model = CatBoostClassifier().load_model(full_path)
    params = model.get_params()
    threshold = 0.5
    model.set_probability_threshold(threshold)
    name = path.split('.')[0][9:]
    average='weighted'
    params['name'] = name
    params['threshold'] = threshold
    test_pred = model.predict(X_test[model.feature_names_])
    print('Взвешанная, все 0', f1_score(Y_test, test_pred*0, average=average))
    print('Невзвешанная, все 0',f1_score(Y_test, test_pred*0))
    print('Взвешанная, все 1',f1_score(Y_test, test_pred*0+1, average=average))
    print('Невзвешанная, все 1',f1_score(Y_test, test_pred*0+1))
    continue
    wandb.init(
        project="WB-reviews-classification",
        name = name,
        tags=[average, 'catboost'],
        config=params, 
    )
    wandb.log({
        'test_f1': f1_score(Y_test, test_pred, average=average),
        'test_precision': precision_score(Y_test, test_pred, average=average),
        'test_recall': recall_score(Y_test, test_pred, average=average),
    })
    wandb.finish()

./catboost_baseline.model
Взвешанная, все 0 0.624073732718894
Невзвешанная, все 0 0.0
Взвешанная, все 1 0.11027848101265822
Невзвешанная, все 1 0.4177215189873418
./catboost_full.model
Взвешанная, все 0 0.624073732718894
Невзвешанная, все 0 0.0
Взвешанная, все 1 0.11027848101265822
Невзвешанная, все 1 0.4177215189873418
./catboost_with_text.model
Взвешанная, все 0 0.624073732718894
Невзвешанная, все 0 0.0
Взвешанная, все 1 0.11027848101265822
Невзвешанная, все 1 0.4177215189873418
./catboost_with_id.model
Взвешанная, все 0 0.624073732718894
Невзвешанная, все 0 0.0
Взвешанная, все 1 0.11027848101265822
Невзвешанная, все 1 0.4177215189873418
./catboost_with_text-class.model
Взвешанная, все 0 0.624073732718894
Невзвешанная, все 0 0.0
Взвешанная, все 1 0.11027848101265822
Невзвешанная, все 1 0.4177215189873418
./catboost_with_text_text-class.model
Взвешанная, все 0 0.624073732718894
Невзвешанная, все 0 0.0
Взвешанная, все 1 0.11027848101265822
Невзвешанная, все 1 0.4177215189873418
./catbo

In [17]:
path = 'catboost_with_text_text-class_tuned.model'
full_path = r"./"+path
print(full_path)
model = CatBoostClassifier().load_model(full_path)
params = model.get_params()
threshold = 0.5
model.set_probability_threshold(threshold)
name = path.split('.')[0][9:]
average='weighted'
params['name'] = name
params['threshold'] = threshold
test_pred = model.predict(X_test[model.feature_names_])
print(f1_score(Y_test, test_pred, average='weighted'))
print(precision_score(Y_test, test_pred, average=average))
wandb.init(
    project="WB-reviews-classification",
    name = name,
    tags=[average, 'catboost', 'tuned'],
    config=params, 
)
wandb.log({
    'test_f1': f1_score(Y_test, test_pred, average=average),
    'test_precision': precision_score(Y_test, test_pred, average=average),
    'test_recall': recall_score(Y_test, test_pred, average=average),
})
wandb.finish()

./catboost_with_text_text-class_tuned.model
0.7697737142857143
0.7858354561101549


## Кастомный класс

In [8]:
class FakeReviewsClassifier:
    def __init__(self, model_path='./catboost_full.model', get_features=True):
        self.catboost_model: CatBoostClassifier = CatBoostClassifier().load_model(model_path)
        self.feature_names_ = self.catboost_model.feature_names_
        self.get_features = get_features
        
    def get_params(self):
        return self.catboost_model.get_params()

    def get_features(self, data: pd.Series):
        return get_features(data)

    def preproccess(self, data: pd.Series, get_features):
        data = data.drop(['f2', 'f4'], errors='ignore')
        if get_features:
            data = self.get_features(data)
            id_data = data.get(['id1_pred', 'id1_in_database', 'id2_pred', 'id2_in_database'])
            data = data.drop(['id1', 'id2', 'id3', 'text', 'label'])   
        else:
            id_data = data.get(['id1_pred', 'id1_in_database', 'id2_pred', 'id2_in_database'])
            data = data[self.feature_names_]
        return data, id_data
    
    # предсказание по id
    def id_predict(self, id_data: pd.Series, threshold):
        if not ('id1_in_database' in id_data.index):
            return None
        if id_data['id1_in_database']:
            return 1 if id_data['id1_pred'] >= threshold else 0
        return None
        
    def predict(self, data: pd.Series, 
                get_features=None, 
                id_predict = True,
                threshold = 0.5):
        if get_features is None:
            get_features = self.get_features
        data, id_data = self.preproccess(data, get_features)
        # если пользователь оставляет очень много фейков или наоборот – честных отзывов
        predict = None
        if id_predict and (not id_data is None):
            predict = self.id_predict(id_data, threshold=threshold)
        if predict is None:
            self.catboost_model.set_probability_threshold(threshold)
            predict = self.catboost_model.predict(data)
        return predict


In [67]:
for path in models_path:
    full_path = r"./"+path
    print(full_path)
    model = FakeReviewsClassifier(full_path)
    params = model.get_params()
    name = path.split('.')[0][9:]
    threshold = 0.5
    average='weighted'
    params['name'] = name
    params['threshold'] = threshold
    test_pred = X_test[model.feature_names_].apply(lambda x: model.predict(x, get_features=False, threshold=threshold), axis=1)
    print(f1_score(Y_test, test_pred, average=average))
    wandb.init(
        # set the wandb project where this run will be logged
        project="WB-reviews-classification",
        name = name+'_custom',
        tags=[average, 'custom', 'catboost'],
        # track hyperparameters and run metadata
        config=params, 
    )
    wandb.log({
        'test_f1': f1_score(Y_test, test_pred, average=average),
        'test_precision': precision_score(Y_test, test_pred, average=average),
        'test_recall': recall_score(Y_test, test_pred, average=average),
    })
    wandb.finish()

./catboost_baseline.model
0.6552185005798484
./catboost_full.model
0.7591067304984511
./catboost_with_text.model
0.6244086361242759
./catboost_with_id.model
0.6389769816344933
./catboost_with_text-class.model
0.7525990515960083
./catboost_with_text_text-class.model
0.7745779672706752
./catboost_with_text.model
0.6244086361242759


In [70]:
for path in models_path:
    full_path = r"./"+path
    print(full_path)
    model = FakeReviewsClassifier(full_path)
    params = model.get_params()
    name = path.split('.')[0][9:]
    threshold = 0.55
    average='weighted'
    params['name'] = name
    params['threshold'] = threshold
    test_pred = X_test[model.feature_names_].apply(lambda x: model.predict(x, get_features=False, threshold=threshold), axis=1)
    print(f1_score(Y_test, test_pred, average=average))
    wandb.init(
        # set the wandb project where this run will be logged
        project="WB-reviews-classification",
        name = name+'_custom',
        tags=[average, 'custom', 'catboost'],
        # track hyperparameters and run metadata
        config=params, 
    )
    wandb.log({
        'test_f1': f1_score(Y_test, test_pred, average=average),
        'test_precision': precision_score(Y_test, test_pred, average=average),
        'test_recall': recall_score(Y_test, test_pred, average=average),
    })
    wandb.finish()

./catboost_baseline.model
0.6812413758548989
./catboost_full.model
0.7746631475755976
./catboost_with_text.model
0.6278301693860312
./catboost_with_id.model
0.6812178807406802
./catboost_with_text-class.model
0.7714258648711401
./catboost_with_text_text-class.model
0.785224236485383
./catboost_with_text.model
0.6278301693860312


In [30]:
path = 'catboost_full_tuned.model'
full_path = r"./"+path
print(full_path)
model = FakeReviewsClassifier(full_path)
params = model.get_params()
name = path.split('.')[0][9:]
threshold = 0.6
average='weighted'
params['name'] = name
params['threshold'] = threshold
test_pred = X_test[model.feature_names_].apply(lambda x: model.predict(x, get_features=False, threshold=threshold), axis=1)
print(f1_score(Y_test, test_pred, average=average))
print(precision_score(Y_test, test_pred, average=average))
print(precision_score(Y_test, test_pred))
wandb.init(
    # set the wandb project where this run will be logged
    project="WB-reviews-classification",
    name = name+'_custom',
    tags=[average, 'custom', 'catboost', 'tuned'],
    # track hyperparameters and run metadata
    config=params,
)
wandb.log({
    'test_f1': f1_score(Y_test, test_pred, average=average),
    'test_precision': precision_score(Y_test, test_pred, average=average),
    'test_recall': recall_score(Y_test, test_pred, average=average),
})
wandb.finish()

./catboost_full_tuned.model
0.8004057275417966
0.799376791643048
0.6289308176100629


SyntaxError: 'continue' not properly in loop (1347988152.py, line 15)

In [66]:
path = 'catboost_full_tuned.model'
full_path = r"./"+path
model = FakeReviewsClassifier(full_path)
test_pred = X_test[model.feature_names_].apply(lambda x: model.predict(x, get_features=False, threshold=threshold), axis=1)

weights = (1/(sum(Y_test==0)*len(Y_test)), 1/(sum(Y_test==1)*len(Y_test)))
y = Y_test.copy()
y.loc[y==0] = weights[0]
y.loc[y==1] = weights[1]

# модель
print('Модель')
print(f1_score(Y_test, test_pred))
print(precision_score(Y_test, test_pred))
print(recall_score(Y_test, test_pred))

print(f1_score(Y_test, test_pred, average='weighted'))
print(precision_score(Y_test, test_pred, average='weighted'))
print(recall_score(Y_test, test_pred, average='weighted'))

print(balanced_accuracy_score(Y_test, test_pred))

# все 0
print('\nВсе 0')
print(f1_score(Y_test, test_pred*0))
print(precision_score(Y_test, test_pred*0))
print(recall_score(Y_test, test_pred*0))

print(f1_score(Y_test, test_pred*0, average='weighted'))
print(precision_score(Y_test, test_pred*0, average='weighted'))
print(recall_score(Y_test, test_pred*0, average='weighted'))

print(balanced_accuracy_score(Y_test, test_pred*0))

# все 1
print('\nВсе 1')
print(f1_score(Y_test, test_pred*0+1))
print(precision_score(Y_test, test_pred*0+1))
print(recall_score(Y_test, test_pred*0+1))

print(f1_score(Y_test, test_pred*0+1, average='weighted'))
print(precision_score(Y_test, test_pred*0+1, average='weighted'))
print(recall_score(Y_test, test_pred*0+1, average='weighted'))

print(balanced_accuracy_score(Y_test, test_pred*0+1))

# рандом
print('\nРандом')
import numpy as np
test_pred = np.random.randint(0, 2, len(Y_test))
print(f1_score(Y_test, test_pred))
print(precision_score(Y_test, test_pred))
print(recall_score(Y_test, test_pred))

print(f1_score(Y_test, test_pred, average='weighted'))
print(precision_score(Y_test, test_pred, average='weighted'))
print(recall_score(Y_test, test_pred, average='weighted'))

print(balanced_accuracy_score(Y_test, test_pred))

Модель
0.6042780748663102
0.5406698564593302
0.6848484848484848
0.7711823798012356
0.7867368421052632
0.7632
0.7380764163372859

Все 0
0.0
0.0
0.0
0.624073732718894
0.541696
0.736
0.5

Все 1
0.4177215189873418
0.264
1.0
0.11027848101265822
0.06969600000000001
0.264
0.5

Рандом
0.28813559322033894
0.22149837133550487
0.4121212121212121
0.4942066140908892
0.5699724253784851
0.4624
0.4462779973649539


c:\Games\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Games\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Games\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
accuracy_score()